In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Sun Feb  9 12:01:18 2020

@author: colin
"""

import nltk
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)


In [28]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['morning', 'everyone', 'with', 'me'])

# Import Dataset
df = pd.read_csv('transcripts_sent (staples combined).csv')
df.head()

,Unnamed: 0,Sent
0,0,"Thank you, Kate, and good morning, everyone, a..."
1,1,With me to review Kroger’s second quarter 2017...
2,2,"As you know, Kroger has competed in an ever-ch..."
3,3,"Our success over time boils down to one thing,..."
4,4,That focus allows us to deepen our connection ...


In [17]:
# Convert to list
data = df.Sent.values.tolist()

pprint(data[:1])

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])


['Thank you, Kate, and good morning, everyone, and thank you for joining us']
[['thank', 'you', 'kate', 'and', 'good', 'morning', 'everyone', 'and', 'thank', 'you', 'for', 'joining', 'us']]


In [29]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=50) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

['thank', 'you', 'kate', 'and', 'good_morning_everyone', 'and', 'thank', 'you', 'for', 'joining_us']


In [30]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['thank', 'thank']]


In [31]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 2)]]


In [32]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]


[(0,
  '0.223*"consumer" + 0.159*"high" + 0.074*"would" + 0.074*"number" + '
  '0.071*"focus" + 0.044*"environment" + 0.036*"become" + 0.030*"food" + '
  '0.029*"dynamic" + 0.028*"side"'),
 (1,
  '0.114*"good" + 0.113*"say" + 0.097*"include" + 0.094*"retail" + '
  '0.093*"time" + 0.069*"price" + 0.062*"change" + 0.054*"remain" + '
  '0.042*"base" + 0.033*"quality"'),
 (2,
  '0.233*"today" + 0.000*"call" + 0.000*"financial_measure" + 0.000*"refer" + '
  '0.000*"acquisition" + 0.000*"add" + 0.000*"top_line" + 0.000*"like" + '
  '0.000*"set" + 0.000*"beauty"'),
 (3,
  '0.546*"brand" + 0.112*"improve" + 0.056*"team" + 0.032*"leverage" + '
  '0.027*"sell" + 0.026*"research" + 0.025*"show" + 0.020*"revenue" + '
  '0.020*"competitor" + 0.013*"ongoing"'),
 (4,
  '0.179*"first" + 0.131*"performance" + 0.105*"strategy" + 0.069*"expand" + '
  '0.057*"core" + 0.044*"operate" + 0.042*"may" + 0.039*"announce" + '
  '0.030*"element" + 0.022*"introduce"'),
 (5,
  '0.193*"increase" + 0.066*"create" + 0

In [33]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)



Perplexity:  -17.551135363516302

Coherence Score:  0.3577987376390176


In [34]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
19    -0.148900 -0.065329       1        1  9.697668
18    -0.172039 -0.248367       2        1  9.083177
6     -0.183274  0.372641       3        1  8.962387
11    -0.103392 -0.012774       4        1  8.536150
9     -0.148588 -0.065046       5        1  8.315017
1     -0.073698 -0.005589       6        1  7.473940
14    -0.057694 -0.003439       7        1  6.375855
0     -0.040385 -0.001385       8        1  5.832724
5     -0.025364  0.000150       9        1  5.764871
15    -0.029181 -0.000785      10        1  4.889009
8      0.002409  0.001245      11        1  4.668907
4      0.027637  0.002735      12        1  4.557684
3     -0.034688 -0.001092      13        1  4.388824
7      0.011463  0.001765      14        1  4.184670
13     0.091032  0.004275      15        1  2.866877
12     0.129738  0.004701      16        1  2.085378
10     0.141999  0.004743      17        1  1.705194
16     0.203157  0.004313      18        1  0.326387
2      0.204619  0.003785      19        1  0.222229
17     0.205147  0.003452      20        1  0.063053, topic_info=             Term         Freq        Total Category  logprob  loglift
212         brand  1869.000000  1869.000000  Default  30.0000  30.0000
14           year  2067.000000  2067.000000  Default  29.0000  29.0000
56       continue  1835.000000  1835.000000  Default  28.0000  28.0000
41         growth  2173.000000  2173.000000  Default  27.0000  27.0000
123      business  1921.000000  1921.000000  Default  26.0000  26.0000
...           ...          ...          ...      ...      ...      ...
1546      evening     0.010472     1.666131  Topic20  -8.4545   2.2994
1547         bean     0.010472     1.666131  Topic20  -8.4545   2.2994
1549  consumption     0.010472     1.666314  Topic20  -8.4545   2.2993
1556      density     0.010472     1.666143  Topic20  -8.4545   2.2994
1550  correlation     0.010472     1.666132  Topic20  -8.4545   2.2994

[876 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
121       1  0.986585     ability
109       1  0.992753  accelerate
308       9  0.978019     account
305       6  0.987106    addition
354       3  0.984149     address
...     ...       ...         ...
62        5  0.998669        well
254      12  0.975111         win
89       15  0.996854        work
388       8  0.995154       would
14        7  0.999209        year

[289 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[20, 19, 7, 12, 10, 2, 15, 1, 6, 16, 9, 5, 4, 8, 14, 13, 11, 17, 3, 18])